In [3]:
from selectorlib import Extractor
import fake_useragent
import requests
import json
# from fake_useragent import UserAgent


# https://www.zyte.com/blog/an-introduction-to-xpath-with-examples/
# In fact, the expressions we've just seen are using XPath's abbreviated syntax. Translating //title to the full syntax we get:
# So, // in the abbreviated syntax is short for descendant-or-self, which means the current node or any node below it in the tree.
# /descendant-or-self::node()/child::title

#//li[position() = 1]
# (notice that positions in XPath start at 1, not 0). We can abbreviate the expression above to:

# //li[1]

# Expression	        Meaning
# //li[position()%2=0]	        Selects the li elements at even positions.
# //li[a]	                    Selects the li elements that enclose an a element.
# //li[a or h2]	                Selects the li elements that enclose either an a or an h2 element.
# //li[ a [ text() = "link" ] ]	Selects the li elements that enclose an a element whose text is "link". Can also be written as //li[ a/text()="link" ].
# //li[last()]	                Selects the last li element in the document.







In [ ]:
# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('products.yml')

def scrape(url):

    ua = UserAgent()


    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)
# End of function



In [ ]:
with open("product_urls.txt",'r') as urllist, open('product_output.jsonl','w') as outfile:
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            try:
                data['seller_link'] = 'https://www.amazon.com' + data['seller_link']
                data['freq_bought_link'] = 'https://www.amazon.com' + data['freq_bought_link']
                json.dump(data,outfile)
                outfile.write("\n")

            except:
                json.dump(data,outfile)
                outfile.write("\n")